# Assignment - classification

Hi there! In this assignment, you will use a fully connected neural network (FCNN) to solve an adapted Question 1 of the winter 2023 exam in applied machine learning:

As in Assignment 1, the primary objective of this exam is to perform image classification using the PCam dataset. For a detailed description of the dataset, please refer to the assignment 1 description. The assignment is posted as a Kaggle competition and is available here: https://www.kaggle.com/t/cda2949c5097437581cdb9abd32091ae

To get you started, I have provided a complete working example, which is decent but not very impressive.

When you are done, submit your results on the Kaggle webpage for this competition. If you do not like to show your score to everyone, you may use an anonymous username on Kaggle.

However, I suggest you use your real name, after all it is just meant as an exercise and it is more fun that way. You can submit 5 times every day, so you can experiment with some stuff without being "locked in".

# Details

The metric used to score this assignment is accuracy (as in the first assignment).

### Question (adapted from the exam):
Use FCNN to perform image classification (tumor detection). Consider among other things the following:
1. Different activation functions
2. Different number of layers
3. Different number of neurons in each layer
4. Different learning rates
5. Different batch sizes
6. Different number of epochs
7. Different optimizers

**Note:** When you do hyperparameter tuning, you should use the validation set. The test set should only be used for the final evaluation.


# Hints to get you started (with a very simple model)

In [53]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd

Defining a function that takes a (None,96,96,3) array and turn it into (None, 32,32,1) (grayscale, resize and normalize). This function might also become handy if the original images are too large for your hardware configuration.

In [55]:
def resize_and_normalize_image(image):
    image = tf.image.resize(image,[32,32])
    image = tf.image.rgb_to_grayscale(image)
    return image / 255.0

def convert_sample(data):

# Create a TensorFlow dataset from the training data features
    dataset = tf.data.Dataset.from_tensor_slices(data)

# Define a function to resize each image in the dataset

# Apply the resize function to each image in the dataset
    resized_dataset = dataset.map(resize_and_normalize_image)

# Convert the resized dataset to a NumPy array
    resized_arr = np.array(list(resized_dataset.as_numpy_iterator()))

    return resized_arr

In [65]:
# Load the training data features
X_train_raw = np.load('/mnt/c/Users/cmd/Dropbox/Teaching/amlFall2023/assignments/Xtrain.npy')
print(f'Shape of the raw training data: {X_train_raw.shape}')
X_test_raw = np.load('/mnt/c/Users/cmd/Dropbox/Teaching/amlFall2023/assignments/Xtest.npy')
print(f'Shape of the raw test data: {X_test_raw.shape}')

X_train = convert_sample(X_train_raw)
print(f'Shape the resized training data: {X_train.shape}')

X_test = convert_sample(X_test_raw)
print(f'Shape the resized test data: {X_test.shape}')

y_raw = np.load('/mnt/c/Users/cmd/Dropbox/Teaching/amlFall2023/assignments/ytrain.npy')
y_raw = y_raw.reshape(-1,1) 
print(f'Shape of the raw labels: {y_raw.shape}')


Shape of the raw training data: (26214, 96, 96, 3)
Shape of the raw test data: (1638, 96, 96, 3)
Shape the resized training data: (26214, 32, 32, 1)
Shape the resized test data: (1638, 32, 32, 1)
Shape of the raw labels: (26214, 1)


In [68]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(32,32,1)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax'),
    ])

sgd_opt = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd_opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_raw, epochs=10)

Epoch 1/10
820/820 [==============================] - 5s 6ms/step - loss: 0.6886 - accuracy: 0.5345
Epoch 2/10
820/820 [==============================] - 5s 6ms/step - loss: 0.6816 - accuracy: 0.5565
Epoch 3/10
820/820 [==============================] - 5s 6ms/step - loss: 0.6763 - accuracy: 0.5656
Epoch 4/10
820/820 [==============================] - 4s 5ms/step - loss: 0.6539 - accuracy: 0.6079
Epoch 5/10
820/820 [==============================] - 5s 5ms/step - loss: 0.6392 - accuracy: 0.6307
Epoch 6/10
820/820 [==============================] - 4s 5ms/step - loss: 0.6322 - accuracy: 0.6445
Epoch 7/10
820/820 [==============================] - 4s 5ms/step - loss: 0.6274 - accuracy: 0.6503
Epoch 8/10
820/820 [==============================] - 4s 5ms/step - loss: 0.6207 - accuracy: 0.6592
Epoch 9/10
820/820 [==============================] - 4s 5ms/step - loss: 0.6166 - accuracy: 0.6634
Epoch 10/10
820/820 [==============================] - 4s 5ms/step - loss: 0.6101 - accuracy: 0.6714

The below code makes predictions and then saves them (after checking they are in correct format).

The argmax converts probabilities to specific class predictions.

And finally convert to appropriate $\texttt{.csv}$ for Kaggle submit.

In [69]:
y_test_hat = model.predict(X_test)
y_test_hat = np.argmax(y_test_hat, axis=1)

ytest_hat_pd = pd.DataFrame({
    'Id': list(range(len(y_test_hat))),
    'Predicted': y_test_hat.reshape(-1,),
})

ytest_hat_pd.to_csv('y_test_hat_fcnn.csv', index=False)

52/52 [==============================] - 0s 6ms/step
